In [1]:
import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
import pandas as pd
import datetime
from pyecharts import options as opts
from pyecharts.charts import Line, Timeline, Bar
import os
from service.InfluxService import InfluxdbService
from utils.InfluxTime import InfluxTime

In [3]:
class GetData:
    def __init__(self):
        self.db = InfluxdbService()

    def get_data(self, start, end, measurement, targetcode=None, opcode=None, df=True, keep=None, filter_=None):
        return self.db.query_influx(start, end, measurement, targetcode=targetcode, opcode=opcode, df=df, keep=keep,
                                    filter_=filter_)

In [4]:
getData=GetData()

2023-02-15 17:03:08,526 - INFO - InfluxDB Connected


In [5]:
end=InfluxTime.utc8now()
print(end)

2023-02-15 17:03:10


In [45]:
df=getData.get_data(start="2023-01-01 00:00:00",
                    end="2023-02-15 00:00:00",
                    measurement="opcontractquote",opcode="10004884.XSHG", df=True)

1672502400.0
1676390400.0


In [46]:
df.iloc[40]

AttributeError: 'NoneType' object has no attribute 'iloc'

In [35]:
def process_df(df)->dict:
    dic=df.to_dict('list')
    #dic["_measurement"]=dic["_measurement"][0]
    #dic["targetcode"]=dic["targetcode"][0]
    return dic

In [36]:
def DrawLine(df):
    if type(df)!=dict:
        df=process_df(df)
    line=Line()
    line.add_xaxis(xaxis_data=df["_time"])
    for i in df:
        line.add_yaxis(i,y_axis=df[i])
    
    line.set_series_opts(label_opts=opts.LabelOpts(is_show=False),)
    line.set_global_opts(title_opts=opts.TitleOpts(title=df["_measurement"][0].upper(),pos_right="50%"),
                    legend_opts=opts.LegendOpts(pos_right="10%",pos_top="20%",orient="vertical"),
                    tooltip_opts =opts.TooltipOpts(trigger='axis',axis_pointer_type='cross'),
                    toolbox_opts =opts.ToolboxOpts(is_show=True,orient='horizontal'),
                    datazoom_opts=opts.DataZoomOpts(range_start=95,range_end=100),
                    xaxis_opts=opts.AxisOpts(type_="time"),)
    
    return line

In [37]:
l=DrawLine(df)
l.render_notebook()